In [1]:
import pandas as pd
import numpy as np
import requests
import json
from dotenv import load_dotenv
import os

In [22]:
load_dotenv()
api_key = os.getenv("EIA_API_KEY")

url = "https://api.eia.gov/v2/electricity/rto/region-data/data/"
params = {
    "api_key": api_key,
    "frequency": "hourly",
    "data[0]": "value",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000
}

response = requests.get(url, params=params)
data = response.json()

print(data.keys())  # Should not say "error" now

# Once we know the correct key path:
df = pd.DataFrame(data["response"]["data"])
print(df.head())

dict_keys(['warnings', 'response', 'request', 'apiVersion', 'ExcelAddInVersion'])
          period respondent                             respondent-name type  \
0  2025-08-16T07        AVA                          Avista Corporation   DF   
1  2025-08-16T07       AZPS              Arizona Public Service Company   DF   
2  2025-08-16T07       BANC  Balancing Authority of Northern California   DF   
3  2025-08-16T07       BPAT             Bonneville Power Administration   DF   
4  2025-08-16T07        CAL                                  California   DF   

                   type-name  value    value-units  
0  Day-ahead demand forecast   1265  megawatthours  
1  Day-ahead demand forecast   4860  megawatthours  
2  Day-ahead demand forecast   2331  megawatthours  
3  Day-ahead demand forecast   6038  megawatthours  
4  Day-ahead demand forecast  34319  megawatthours  


In [23]:
# Save to CSV 
df.to_csv("Electricity.csv", index=False)
print("Data saved to Electricity.csv")

Data saved to Electricity.csv


In [2]:
#working locally
df = pd.read_csv("Electricity.csv")

In [3]:
df.head()  

,period,respondent,respondent-name,type,type-name,value,value-units
0,2025-08-16T07,AVA,Avista Corporation,DF,Day-ahead demand forecast,1265,megawatthours
1,2025-08-16T07,AZPS,Arizona Public Service Company,DF,Day-ahead demand forecast,4860,megawatthours
2,2025-08-16T07,BANC,Balancing Authority of Northern California,DF,Day-ahead demand forecast,2331,megawatthours
3,2025-08-16T07,BPAT,Bonneville Power Administration,DF,Day-ahead demand forecast,6038,megawatthours
4,2025-08-16T07,CAL,California,DF,Day-ahead demand forecast,34319,megawatthours
